In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# ETL Urbania 2

In [2]:
df = pd.read_csv('./dataset/01_etl_urbania.csv')
df.rename(columns={'antiquity': 'year', 'total_bath': 'bath', 'total_bed': 'room'}, inplace=True)
print(df.shape)
df.head(4)

(16697, 14)


,title,address,price,area,bath,room,property_type,description,parking,year,general_features,services,ambience,property_url
0,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",120375.0,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,"{""Promotion Description"",""Promotion Title"",""Nú...","{""Areas verdes""}",NaN,https://urbania.pe/inmueble/proyecto/ememvein-...
1,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",119277.0,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,"{""Promotion Description"",""Promotion Title"",""Nú...","{""Areas verdes""}",NaN,https://urbania.pe/inmueble/proyecto/ememvein-...
2,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",162297.0,60,2,2,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,"{""Promotion Description"",""Promotion Title"",""Nú...","{""Areas verdes""}",NaN,https://urbania.pe/inmueble/proyecto/ememvein-...
3,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",157815.0,60,2,2,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,"{""Promotion Description"",""Promotion Title"",""Nú...","{""Areas verdes""}",NaN,https://urbania.pe/inmueble/proyecto/ememvein-...


## Property Type

In [3]:
df = df[~(df['property_type'].isna())]
df['description'].fillna('', inplace=True)
df['general_features'].fillna('{}', inplace=True)
df['services'].fillna('{}', inplace=True)
df['ambience'].fillna('{}', inplace=True)
print(df.shape)
df[['general_features', 'services', 'ambience']].isna().sum()

(16696, 14)


general_features    0
services            0
ambience            0
dtype: int64

In [4]:
df['property_type'].value_counts()

property_type
Departamento     12606
Casa              3738
Casa de playa      250
Casa de campo      100
Habitación           2
Name: count, dtype: int64

## General Features

The 'general_features' field contains multiple features houses can have, for example, being near to parks, supermarkets, pricipal streets, schools and so forth.

Let's see what is the most common features

In [6]:
general_features = df['general_features'].str.strip('{}').str.split(',').to_list()
general_features = [x.strip('""') for y in general_features for x in y]
genereal_counter = Counter(general_features)
genereal_counter = dict(sorted(genereal_counter.items(), key=lambda item: item[1], reverse=True))

for element, count in genereal_counter.items():
    if count <= df.shape[0]*0.2:
        break
    print(f"{element}: {count}")

Número de pisos: 10064
Cerca a colegios: 6335
Centros Comerciales Cercanos: 6190
Reposteros en cocina: 5297
Cocina: 5123
Ascensor: 4978
Terraza: 4709
Intercomunicador: 4699
Closet: 4657
Cuartos de servicio: 4585
Cerca a Parque (a menos de 2 cdras): 4438
Baño de servicio: 4320
Seguridad: 4098
Mascotas: 4060
Piso en el que se encuentra: 3731


In [7]:
df['near_cc'] = np.zeros(df.shape[0], dtype=int)
df['near_school'] = np.zeros(df.shape[0], dtype=int)
df['near_parks'] = np.zeros(df.shape[0], dtype=int)

### Near to Malls

Let's create a new column that says whether or not the house is nearby malls, depicted by 0 and 1.

In [8]:
mask = df['general_features'].apply(lambda x: 'Centros Comerciales Cercanos' in x)
df.loc[mask, 'near_cc'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('0 :', df[df['near_cc'] == 0].shape[0])
print('1 :', df[df['near_cc'] == 1].shape[0])

0 : 10506
1 : 6190


On the other hand, maybe that info could be into the description. Let's check out

In [9]:
pattern = 'mall|comercial|mercado'
id_mall = df.loc[df['near_cc'] == 0]['description'].apply(lambda x: x.lower()).str.extract(f'({pattern})').dropna().index
df.loc[id_mall, 'near_cc'] = np.ones(shape=df.loc[id_mall].shape[0], dtype=int)
print('0 :', df[df['near_cc'] == 0].shape[0])
print('1 :', df[df['near_cc'] == 1].shape[0])


0 : 6620
1 : 10076


### Near to Schools

In [10]:
mask = df['general_features'].apply(lambda x: 'Cerca a colegios' in x)
df.loc[mask, 'near_school'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('0 :', df[df['near_school'] == 0].shape[0])
print('1 :', df[df['near_school'] == 1].shape[0])

0 : 10361
1 : 6335


In [11]:
pattern = 'colegio|escuela|educativ'
id_school = df.loc[df['near_school'] == 0]['description'].apply(lambda x: x.lower()).str.extract(f'({pattern})').dropna().index
df.loc[id_school, 'near_school'] = np.ones(shape=df.loc[id_school].shape[0], dtype=int)
print('0 :', df[df['near_school'] == 0].shape[0])
print('1 :', df[df['near_school'] == 1].shape[0])

0 : 8004
1 : 8692


### Near to Parks

Into this field we take into account the parks, green areas, garden, see, pool and gyms.

In [15]:
mask = df['general_features'].apply(lambda x: 'Parque' in x)
df.loc[mask, 'near_parks'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('0 :', df[df['near_parks'] == 0].shape[0])
print('1 :', df[df['near_parks'] == 1].shape[0])

0 : 11271
1 : 5425


### Extra Features

Instead of looking for nearby places, let's search if we could get data about elevators, security, gardens or green areas and so forth.

In [16]:
df['security'] = np.zeros(df.shape[0], dtype=int)
df['elevator'] = np.zeros(df.shape[0], dtype=int)
df['rest_area'] = np.zeros(df.shape[0], dtype=int)
df['pool'] = np.zeros(df.shape[0], dtype=int)

In [33]:
# Security
security_list = ['Seguridad', 'Caseta de guardia']
mask = df['general_features'].apply(lambda x: any(s in x for s in security_list))
df.loc[mask, 'security'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Security')
print('0 :', df[df['security'] == 0].shape[0])
print('1 :', df[df['security'] == 1].shape[0])

# Elevator
mask = df['general_features'].apply(lambda x: 'Ascensor' in x)
df.loc[mask, 'elevator'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Elevator')
print('0 :', df[df['elevator'] == 0].shape[0])
print('1 :', df[df['elevator'] == 1].shape[0])

# Rest Area
rest_area_list = ['Terraza', 'Jardín']
mask = df['general_features'].apply(lambda x: any(s in x for s in rest_area_list))
df.loc[mask, 'rest_area'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Rest Area')
print('0 :', df[df['rest_area'] == 0].shape[0])
print('1 :', df[df['rest_area'] == 1].shape[0])

# Pool
pool_list = ['Piscina', 'Jacuzzi']
mask = df['general_features'].apply(lambda x: any(s in x for s in pool_list))
df.loc[mask, 'pool'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Pool')
print('0 :', df[df['pool'] == 0].shape[0])
print('1 :', df[df['pool'] == 1].shape[0])

df[mask].shape

Security
0 : 12225
1 : 4471
Elevator
0 : 11718
1 : 4978
Rest Area
0 : 11161
1 : 5535
Pool
0 : 14063
1 : 2633


(2633, 21)

# Services

As well as general features, let's what kind of thing this Field provides us

In [34]:
services = df['services'].str.strip('{}').str.split(',').to_list()
services = [x.strip('""') for y in services for x in y]
service_counter = Counter(services)
service_counter = dict(sorted(service_counter.items(), key=lambda item: item[1], reverse=True))

for element, count in service_counter.items():
    if count <= df.shape[0]*0.2:
        break
    print(f"{element}: {count}")

Area de lavandería: 7603
Servicios básicos (agua/luz): 6793
: 5186
Guardianía/Seguridad privada: 4670
Parrilla: 4056
Areas verdes: 3627


From this fields let's take into account 'Area de lavanderia' and 'Seguridad privada'

### Extra Features from Services

In [38]:
# Security
security_list = ['Seguridad privada', 'Video vigilancia']
mask = df['services'].apply(lambda x: any(s in x for s in security_list))
df.loc[mask, 'security'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Security')
print('0 :', df[df['security'] == 0].shape[0])
print('1 :', df[df['security'] == 1].shape[0])

# Rest Area
rest_area_list = ['Parrilla', 'Areas verdes']
mask = df['services'].apply(lambda x: any(s in x for s in rest_area_list))
df.loc[mask, 'rest_area'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Rest Area')
print('0 :', df[df['rest_area'] == 0].shape[0])
print('1 :', df[df['rest_area'] == 1].shape[0])

Security
0 : 10513
1 : 6183
Rest Area
0 : 8679
1 : 8017


# Ambience

Once again, let's see what field can help us to provide more info about the houses.

In [39]:
ambience = df['ambience'].str.strip('{}').str.split(',').to_list()
ambience = [x.strip('""') for y in ambience for x in y]
ambience_counter = Counter(ambience)
ambience_counter = dict(sorted(ambience_counter.items(), key=lambda item: item[1], reverse=True))

for element, count in ambience_counter.items():
    if count <= df.shape[0]*0.1:
        break
    print(f"{element}: {count}")

: 9992
Dormitorio principal con baño: 4419
Desagüe: 2361
Sala de estar: 2047
Comedor Diario: 1972
Patio: 1885
Hall de ingreso: 1820


In [40]:
Counter(ambience)

Counter({'': 9992,
         'Dormitorio principal con baño': 4419,
         'Desagüe': 2361,
         'Sala de estar': 2047,
         'Comedor Diario': 1972,
         'Patio': 1885,
         'Hall de ingreso': 1820,
         'Area común': 1409,
         'Ingreso independiente': 1292,
         'Sala de entretenimiento': 1130,
         'Cocheras de visitas': 845,
         'Parque Interno': 626,
         'Equipado': 616,
         'Bodega(s)': 588,
         'Club House': 459,
         'Sauna': 285,
         'Solarium': 110,
         'Turco': 44,
         'Cerco eléctrico': 14,
         'Sala': 5,
         'Area deportiva': 4,
         'Deposito': 3,
         'Oficinas': 3,
         'Baño Propio': 2,
         'Salón de usos múltiples': 2,
         'Atico': 2,
         'Escritorio': 2,
         'Guardería': 2,
         'Sótano': 1,
         'Area de cafetería': 1})

In [45]:
# Rest Area
mask = df['ambience'].apply(lambda x: 'Patio' in x)
df.loc[mask, 'rest_area'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Rest Area')
print('0 :', df[df['rest_area'] == 0].shape[0])
print('1 :', df[df['rest_area'] == 1].shape[0])

# Pool
mask = df['ambience'].apply(lambda x: 'Sauna' in x)
df.loc[mask, 'pool'] = np.ones(shape=df[mask].shape[0], dtype=int)
print('Pool')
print('0 :', df[df['pool'] == 0].shape[0])
print('1 :', df[df['pool'] == 1].shape[0])

Rest Area
0 : 8378
1 : 8318
Pool
0 : 14041
1 : 2655


## Principal Streets

Another notable variable is the house is near to principal streets. However, as we have seen there is not any data about principla streets in the field of 'general_features', 'services' and 'ambience'. Then let's search into the description

In [46]:
df['near_avenue'] = np.zeros(shape=df.shape[0], dtype=int)
df.head(3)

,title,address,price,area,bath,room,property_type,description,parking,year,...,ambience,property_url,near_cc,near_school,near_parks,security,elevator,rest_area,pool,near_avenue
0,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",120375.0,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,...,{},https://urbania.pe/inmueble/proyecto/ememvein-...,1,0,1,0,0,1,0,0
1,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",119277.0,45,2,3,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,...,{},https://urbania.pe/inmueble/proyecto/ememvein-...,1,0,1,0,0,1,0,0
2,Proyecto inmobilario en venta en Calle Coronel...,"Calle Coronel Odriozola 128, San IsidroLima, Lima",162297.0,60,2,2,Departamento,Mozz se deriva de la palabra en italiano mozio...,1,2023,...,{},https://urbania.pe/inmueble/proyecto/ememvein-...,1,0,1,0,0,1,0,0


In [81]:
pattern = '(cerca|acceso|rápid|salida|principal|concurrid|important|paso)[\w*\s*]*avenida.*'
id_street = df['description'].apply(lambda x: x.lower()).str.extract(f'({pattern})').dropna().index
df.loc[id_street, 'near_avenue'] = np.ones(shape=len(id_street), dtype=int)
print('Principal Streets')
print('0 :', df[df['near_avenue'] == 0].shape[0])
print('1 :', df[df['near_avenue'] == 1].shape[0])

Principal Streets
0 : 14842
1 : 1854


## Save Data

In [83]:
df.columns

Index(['title', 'address', 'price', 'area', 'bath', 'room', 'property_type',
       'description', 'parking', 'year', 'general_features', 'services',
       'ambience', 'property_url', 'near_cc', 'near_school', 'near_parks',
       'security', 'elevator', 'rest_area', 'pool', 'near_avenue'],
      dtype='object')

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16696 entries, 0 to 16696
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             16696 non-null  object 
 1   address           16696 non-null  object 
 2   price             16696 non-null  float64
 3   area              16696 non-null  int64  
 4   bath              16696 non-null  int64  
 5   room              16696 non-null  int64  
 6   property_type     16696 non-null  object 
 7   description       16696 non-null  object 
 8   parking           16696 non-null  int64  
 9   year              16696 non-null  int64  
 10  general_features  16696 non-null  object 
 11  services          16696 non-null  object 
 12  ambience          16696 non-null  object 
 13  property_url      16696 non-null  object 
 14  near_cc           16696 non-null  int64  
 15  near_school       16696 non-null  int64  
 16  near_parks        16696 non-null  int64  
 17

In [85]:
df[
    ['title', 'address', 'price', 'area', 'bath', 'room', 'property_type',
    'description', 'year', 'parking', 'near_cc', 'near_school', 'near_parks',
    'near_avenue','security','elevator','rest_area','pool']
].to_csv('./dataset/urbania_final.csv', index=False)

# Concat All Data

In [89]:
df_urbania = pd.read_csv('./dataset/urbania_final.csv')
df_urbania['web'] = ['Urbania'] * df_urbania.shape[0]
df_urbania.columns

Index(['title', 'address', 'price', 'area', 'bath', 'room', 'property_type',
       'description', 'year', 'parking', 'near_cc', 'near_school',
       'near_parks', 'near_avenue', 'security', 'elevator', 'rest_area',
       'pool', 'web'],
      dtype='object')

In [90]:
df_mitula = pd.read_csv('./dataset/mitula_final.csv')
df_mitula.columns

Index(['title', 'address', 'price', 'area', 'bath', 'room', 'property_type',
       'description', 'year', 'parking', 'near_cc', 'near_school',
       'near_parks', 'near_avenue', 'security', 'elevator', 'rest_area',
       'pool', 'web'],
      dtype='object')

In [92]:
df_trovit = pd.read_csv('./dataset/trovit_final.csv')
df_trovit['web'] = ['Trovit'] * df_trovit.shape[0]
df_trovit.columns

Index(['title', 'address', 'price', 'area', 'bath', 'room', 'property_type',
       'description', 'year', 'parking', 'near_cc', 'near_school',
       'near_parks', 'near_avenue', 'security', 'elevator', 'rest_area',
       'pool', 'web'],
      dtype='object')

In [111]:
df_final = pd.concat([df_urbania, df_mitula, df_trovit])
df_final['price'] = df_final['price'].astype(int)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26912 entries, 0 to 5296
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          26912 non-null  object
 1   address        26912 non-null  object
 2   price          26912 non-null  int64 
 3   area           26912 non-null  int64 
 4   bath           26912 non-null  int64 
 5   room           26912 non-null  int64 
 6   property_type  26912 non-null  object
 7   description    26910 non-null  object
 8   year           26912 non-null  int64 
 9   parking        26912 non-null  int64 
 10  near_cc        26912 non-null  int64 
 11  near_school    26912 non-null  int64 
 12  near_parks     26912 non-null  int64 
 13  near_avenue    26912 non-null  int64 
 14  security       26912 non-null  int64 
 15  elevator       26912 non-null  int64 
 16  rest_area      26912 non-null  int64 
 17  pool           26912 non-null  int64 
 18  web            26912 non-null  o

In [112]:
df_final.to_csv('./dataset/final_data.csv', index=False)

In [115]:
df_final['title'].apply(lambda x: len(x)).max()

1325

## Into Postgres

In [120]:
import psycopg2
from sqlalchemy import create_engine
from local_settings import hostname, username, password, port, database

In [121]:
connection = psycopg2.connect(
            host=hostname, user=username, 
            password=password, dbname=database, 
            port=port
        )

engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

In [122]:
# DO NOT UNCOMMENT UNLEES YOU KNOW WHAT YOU ARE DOING
# df_final.to_sql('data_raw', engine, if_exists='replace', index=False)

912

In [123]:
connection.close()